In [ ]:
!sh get_data.sh

In [ ]:
import numpy as np
from scipy.special import factorial, softmax
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from load_data import get_data

In [ ]:
from buffet import IndianBuffet

def init_ibm(data):
    Xs = len(data)
    return IndianBuffet(data, (5, 1, 1), [(1.7, 1, 1)] * Xs, [(0.5, 1, 1)] * Xs)

In [ ]:
X_train, X_test = get_data(include_labels=False)

In [ ]:
ibm = init_ibm(X_train)

In [ ]:
ibm.run_sampler()

In [ ]:
def predict(model, Xs):
    best = (-float('inf'), None, None)
    for Z in model.Zs:
        m = Z.sum(axis=0) - Z
        lps = np.sum(np.log(np.where(Z == 1, m, model.N - m) / model.N), axis=1)
        for f, X in zip(model.feats[:-1], Xs[:-1]):
            if X is not None:
                X_bar = Z @ f.weights(Z)
                diffs = np.sum((X - X_bar) ** 2, axis=1) / (2 * f.var_x)
                lps -= diffs + np.log(2 * np.pi) / 2 + np.log(f.var_x) * f.D / 2
        i = np.argmax(lps)
        if lps[i] > best[0]:
            best = lps[i], Z, i
    best_lp, best_Z, best_i = best
    W = model.feats[-1].weights(best_Z)
    X_w_pred = best_Z[best_i] @ W
    
    return Xs[-1][np.argmax(X_w_pred)] == 1

In [ ]:
def get_X_types(X):
    N = len(X[0])
    return {
        'multi': X,
        'vision_uni': [X[0], [None] * N, X[2]],
        'audio_uni': [[None] * N, X[1], X[2]],
        'vision': [X[0], X[2]],
        'audio': [X[1], X[2]],
    }

def run_experiment(X_train, X_test, seed=None):
    N_train = len(X_train[0])
    N_test = len(X_test[0])
    trains = get_X_types(X_train)
    tests = get_X_types(X_test)
    
    train_test_map = {
        'vision': ['vision'],
        'audio': ['audio'],
        'multi': ['multi', 'vision_uni', 'audio_uni']
    }
    ibms = []
    
    for train_type in train_test_map.keys():
        np.random.seed(seed)
        print(f'Training {train_type}...')
        ibm = init_ibm(trains[train_type])
        ibm.run_sampler(iters=5000)
        avg_K = np.mean([Z.shape[1] for Z in ibm.Zs])
        print(f'Average K: {avg_K}')
        
        for test_type in train_test_map[train_type]:
            train_predictions = [predict(ibm, xs) for xs in zip(*trains[test_type])]
            test_predictions = [predict(ibm, xs) for xs in zip(*tests[test_type])]
            
            train_acc = sum(train_predictions) / N_train
            test_acc = sum(test_predictions) / N_test
            
            print(f'TRAIN: {train_type}, TEST: {test_type}', f'train accuracy: {train_acc}', f'test_accuracy: {test_acc}', sep='\n')
        ibms.append(ibm)
    return ibms

In [ ]:
run_experiment(X_train, X_test, seed=7000)

In [ ]:
# Calculate Bhattacharyya distance between p(Z|X_V) & p(Z|X_A)
def batt_dist(model):
    dist = 0
    posts_A = softmax(model.prior() + np.array([model.feats[1].log_likelihood(Z) for Z in model.Zs]))
    posts_V = softmax(model.prior() + np.array([model.feats[0].log_likelihood(Z) for Z in model.Zs]))

    pq = posts_A * posts_V
    dist = np.sum(np.sqrt(pq))

    return - np.log(dist)

# Calculate Bhattacharyya distance between p(Z|X_V) & uniform
def batt_dist_unif(model):
    dist = 0
    
    posts_V = softmax(model.prior() + np.array([model.feats[0].log_likelihood(Z) for Z in model.Zs]))
    
    pq = posts_V / len(model.Zs)
    dist = np.sum(np.sqrt(pq))

    return - np.log(dist)

print(batt_dist(ibm))
print(batt_dist_unif(ibm))